In [1]:
from OpinionDistribution import OpinionDistribution

In [2]:
n_lobbyists = 0

In [3]:
import json
with open(f'../simulations/{n_lobbyists}_lobbyistsconfig.json', 'r') as f:
    params = json.load(f)

In [4]:
print(params)

{'p_o': 0.01, 'p_p': 0.99, 'lambda_values': [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'phi_values': [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'T': 10000, 'n_lobbyists': 0, 'ms': None, 'lobbyists_data': None, 'nruns': 3}


In [5]:
p_o = params['p_o']
p_p = params['p_p']
lambda_values = params['lambda_values']
phi_values = params['phi_values']
n_lobbyists = params['n_lobbyists']
lobbyists_data = params['lobbyists_data']
nruns = params['nruns']

In [6]:
type(lambda_values)

list

In [7]:
type(phi_values)

list

In [8]:
import os

for _, (lambda_v, phi_v) in enumerate([(l, p) for l in lambda_values for p in phi_values]):    
    path = os.path.join(f'../simulations/{n_lobbyists}_lobbyists/', f'{lambda_v}_{phi_v}/')        
    for run in range(nruns):
        runpath = os.path.join(path, str(run))
        with open(runpath+'/status.json', 'r') as f:
            trends = json.load(f)
        final_distribution = trends[-1]
        od = OpinionDistribution(trends, p_o, p_p, values='probabilities')
        od.plot(filename=runpath+'/final_probabilities_distribution.png')
        

